In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

#messing around
import tensorflow as tf

## Loading and Preprocessing Data

In [45]:
import chardet    
rawdata = open(r'SCDB_2021_01_caseCentered_Citation.csv', 'rb').read()
result = chardet.detect(rawdata)
charenc = result['encoding']
print(charenc)

Windows-1252


In [46]:
cases = pd.read_csv(r'SCDB_2021_01_caseCentered_Citation.csv', encoding=charenc)
pd.options.display.max_columns = None
display(cases)
cases.shape

,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,lexisCite,term,naturalCourt,chief,docket,caseName,dateArgument,dateRearg,petitioner,petitionerState,respondent,respondentState,jurisdiction,adminAction,adminActionState,threeJudgeFdc,caseOrigin,caseOriginState,caseSource,caseSourceState,lcDisagreement,certReason,lcDisposition,lcDispositionDirection,declarationUncon,caseDisposition,caseDispositionUnusual,partyWinning,precedentAlteration,voteUnclear,issue,issueArea,decisionDirection,decisionDirectionDissent,authorityDecision1,authorityDecision2,lawType,lawSupp,lawMinor,majOpinWriter,majOpinAssigner,splitVote,majVotes,minVotes
0,1946-001,1946-001-01,1946-001-01-01,1946-001-01-01-01,11/18/1946,1,329 U.S. 1,67 S. Ct. 6,91 L. Ed. 3,1946 U.S. LEXIS 1724,1946,1301,Vinson,24,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,1/9/1946,10/23/1946,198,NaN,172.0,NaN,6.0,NaN,NaN,0.0,51.0,6.0,29.0,NaN,0.0,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,8.0,2.0,0.0,4.0,NaN,6.0,600.0,35 U.S.C. § 33,78.0,78.0,1,8,1
1,1946-002,1946-002-01,1946-002-01-01,1946-002-01-01-01,11/18/1946,1,329 U.S. 14,67 S. Ct. 13,91 L. Ed. 12,1946 U.S. LEXIS 1725,1946,1301,Vinson,12,CLEVELAND v. UNITED STATES,10/10/1945,10/17/1946,100,NaN,27.0,NaN,1.0,NaN,NaN,0.0,123.0,52.0,30.0,NaN,0.0,4.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,10500.0,1.0,1.0,0.0,4.0,NaN,6.0,600.0,18 U.S.C. § 398,81.0,87.0,1,6,3
2,1946-003,1946-003-01,1946-003-01-01,1946-003-01-01-01,11/18/1946,1,329 U.S. 29,67 S. Ct. 1,91 L. Ed. 22,1946 U.S. LEXIS 3037,1946,1301,Vinson,21,CHAMPLIN REFINING CO. v. UNITED STATES ET AL.,11/8/1945,10/18/1946,209,NaN,27.0,NaN,2.0,66.0,NaN,1.0,107.0,42.0,107.0,42.0,0.0,1.0,NaN,2.0,1.0,2.0,0.0,0.0,0.0,0.0,80250.0,8.0,2.0,0.0,1.0,NaN,2.0,207.0,NaN,84.0,78.0,1,5,4
3,1946-004,1946-004-01,1946-004-01-01,1946-004-01-01-01,11/25/1946,7,329 U.S. 40,67 S. Ct. 167,91 L. Ed. 29,1946 U.S. LEXIS 1696,1946,1301,Vinson,26,UNITED STATES v. ALCEA BAND OF TILLAMOOKS ET AL.,1/31/1946,10/25/1946,27,NaN,170.0,NaN,1.0,67.0,NaN,0.0,3.0,NaN,3.0,NaN,0.0,10.0,NaN,2.0,1.0,2.0,0.0,0.0,0.0,0.0,20150.0,2.0,2.0,0.0,4.0,NaN,6.0,600.0,49 Stat. 801,87.0,87.0,1,5,3
4,1946-005,1946-005-01,1946-005-01-01,1946-005-01-01-01,11/25/1946,1,329 U.S. 64,67 S. Ct. 154,91 L. Ed. 44,1946 U.S. LEXIS 2997,1946,1301,Vinson,50,"UNITED STATES v. HOWARD P. FOLEY CO., INC.",10/25/1946,NaN,27,NaN,176.0,NaN,1.0,NaN,NaN,0.0,3.0,NaN,3.0,NaN,0.0,2.0,NaN,2.0,1.0,3.0,0.0,1.0,0.0,0.0,80060.0,8.0,2.0,0.0,7.0,NaN,NaN,NaN,NaN,78.0,87.0,1,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9090,2020-070,2020-070-01,2020-070-01-01,2020-070-01-01-01,3/29/2021,2,NaN,141 S. Ct. 1145,209 L. Ed. 2d 265,2021 U.S. LEXIS 1736,2020,1709,Roberts,20-507,"MAYS, WARDEN v. HINES",NaN,NaN,7,50.0,215.0,NaN,1.0,NaN,NaN,0.0,117.0,NaN,26.0,NaN,1.0,12.0,3.0,2.0,1.0,3.0,0.0,1.0,0.0,0.0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,8,1
9091,2020-071,2020-071-01,2020-071-01-01,2020-071-01-01-01,4/26/2021,2,NaN,141 S. Ct. 1467,209 L. Ed. 2d 431,2021 U.S. LEXIS 2224,2020,1709,Roberts,20-940,ALASKA v. WRIGHT,NaN,NaN,28,2.0,126.0,NaN,1.0,NaN,NaN,0.0,44.0,NaN,29.0,NaN,0.0,12.0,4.0,2.0,1.0,5.0,0.0,1.0,0.0,0.0,10020.0,1.0,1.0,NaN,4.0,3.0,3.0,341.0,NaN,NaN,111.0,1,9,0
9092,2020-072,2020-072-01,2020-072-01-01,2020-072-01-01-01,7/2/2021,2,NaN,141 S. Ct. 2405,210 L. Ed. 2d 812,2021 U.S. LEXIS 3590,2020,1709,Roberts,20-1084,DUNN v. REEVES,NaN,NaN,7,1.0,215.0,NaN,1.0,NaN,NaN,0.0,43.0,NaN,31.0,NaN,0.0,12.0,6.0,2.0,1.0,4.0,0.0,1.0,0.0,0.0,10020.0,1.0,1.0,0.0,3.0,NaN,3.0,341.0,NaN,NaN,111.0,1,6,3
9093,2020-073,2020-073-01,2020-073-01-01,2020-073-01-01-01,6/28/2021,2,NaN,141 S. Ct. 2226,210 L. Ed. 2d 617,2021 U.S. LEXIS 3557,2020,1709,Roberts,20-1212,PAKDEL v. CITY AND COUNTY OF SAN FRANCISCO,NaN,NaN,195,NaN,3.0,6.0,1.0,NaN,NaN,0.0,50.0,NaN,29.0,NaN,1.0,12.0,2.0,2.0,3.0,5.0,0.0,

(9095, 53)

In [47]:
print(cases['chief'].unique());

['Vinson' 'Warren' 'Burger' 'Rehnquist' 'Roberts']


In [48]:
#converting chief justice name to numerical values
"""
Chief Justices:
- Vinson
- Warren
- Burger
- Rehnquist
- Roberts
"""


cases.loc[cases['chief'] == "Vinson", 'chief'] = 1
cases.loc[cases['chief'] == "Warren", 'chief'] = 2
cases.loc[cases['chief'] == "Burger", 'chief'] = 3
cases.loc[cases['chief'] == "Rehnquist", 'chief'] = 4
cases.loc[cases['chief'] == "Roberts", 'chief'] = 5


In [49]:
cases = cases.get(['term', 'naturalCourt', 'petitioner', 'respondent', 'jurisdiction', 'caseOrigin', 'caseSource', 'lcDisagreement', 'certReason', 'lcDispositionDirection','declarationUncon','caseDisposition','caseDispositionUnusual','partyWinning','precedentAlteration','voteUnclear','issue','issueArea','authorityDecision1','majOpinAssigner','splitVote','majVotes','minVotes','chief','decisionDirection'])

cases

,term,naturalCourt,petitioner,respondent,jurisdiction,caseOrigin,caseSource,lcDisagreement,certReason,lcDispositionDirection,declarationUncon,caseDisposition,caseDispositionUnusual,partyWinning,precedentAlteration,voteUnclear,issue,issueArea,authorityDecision1,majOpinAssigner,splitVote,majVotes,minVotes,chief,decisionDirection
0,1946,1301,198,172.0,6.0,51.0,29.0,0.0,11.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,8.0,4.0,78.0,1,8,1,1,2.0
1,1946,1301,100,27.0,1.0,123.0,30.0,0.0,4.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,10500.0,1.0,4.0,87.0,1,6,3,1,1.0
2,1946,1301,209,27.0,2.0,107.0,107.0,0.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,80250.0,8.0,1.0,78.0,1,5,4,1,2.0
3,1946,1301,27,170.0,1.0,3.0,3.0,0.0,10.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,20150.0,2.0,4.0,87.0,1,5,3,1,2.0
4,1946,1301,27,176.0,1.0,3.0,3.0,0.0,2.0,2.0,1.0,3.0,0.0,1.0,0.0,0.0,80060.0,8.0,7.0,87.0,1,6,3,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9090,2020,1709,7,215.0,1.0,117.0,26.0,1.0,12.0,2.0,1.0,3.0,0.0,1.0,0.0,0.0,10020.0,1.0,4.0,111.0,1,8,1,5,1.0
9091,2020,1709,28,126.0,1.0,44.0,29.0,0.0,12.0,2.0,1.0,5.0,0.0,1.0,0.0,0.0,10020.0,1.0,4.0,111.0,1,9,0,5,1.0
9092,2020,1709,7,215.0,1.0,43.0,31.0,0.0,12.0,2.0,1.0,4.0,0.0,1.0,0.0,0.0,10020.0,1.0,3.0,111.0,1,6,3,5,1.0
9093,2020,1709,195,3.0,1.0,50.0,29.0,1.0,12.0,2.0,3.0,5.0,0.0,1.0,0.0,0.0,40070.0,4.0,2.0,111.0,1,9,0,5,1.0


In [50]:
#drop null values in data
cases = cases.dropna()

In [51]:
cases.shape

(8506, 25)

# Keras Deep Learning Multi-class Classification

In [52]:
# multi-class classification with Keras
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


In [53]:
# load dataset
dataset = cases.values
dataset


array([[1946, 1301, 198, ..., 1, 1, 2.0],
       [1946, 1301, 100, ..., 3, 1, 1.0],
       [1946, 1301, 209, ..., 4, 1, 2.0],
       ...,
       [2020, 1709, 7, ..., 3, 5, 1.0],
       [2020, 1709, 195, ..., 0, 5, 1.0],
       [2020, 1709, 202, ..., 3, 5, 2.0]], dtype=object)

In [11]:
dataset.shape

(8506, 25)

In [12]:
X = dataset[:,0:24].astype(float)
Y = dataset[:,24]

In [13]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [68]:
# define baseline model
def baseline_model():
 # create model
 model = Sequential()
 model.add(Dense(100, input_dim=24, activation='relu'))
 model.add(Dense(100, activation='relu'))
 model.add(Dense(100, activation='relu'))
 model.add(Dense(3, activation='softmax'))
 # Compile model
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

In [71]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=30, batch_size=5, verbose=1)
kfold = KFold(n_splits=10, shuffle=True)


/tmp/ipykernel_2144/3817828019.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=baseline_model, epochs=30, batch_size=5, verbose=1)


In [72]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/30
1531/1531 [==============================] - 5s 3ms/step - loss: 109.0584 - accuracy: 0.4956
Epoch 2/30
1531/1531 [==============================] - 4s 3ms/step - loss: 8.0911 - accuracy: 0.5079
Epoch 3/30
1531/1531 [==============================] - 4s 3ms/step - loss: 1.1379 - accuracy: 0.5169
Epoch 4/30
1531/1531 [==============================] - 4s 3ms/step - loss: 0.7254 - accuracy: 0.5087
Epoch 5/30
1531/1531 [==============================] - 5s 3ms/step - loss: 0.7251 - accuracy: 0.5083
Epoch 6/30
1531/1531 [==============================] - 5s 3ms/step - loss: 0.7252 - accuracy: 0.5095
Epoch 7/30
1531/1531 [==============================] - 5s 3ms/step - loss: 0.7257 - accuracy: 0.5069
Epoch 8/30
1531/1531 [==============================] - 5s 3ms/step - loss: 0.7257 - accuracy: 0.5093
Epoch 9/30
1531/1531 [==============================] - 5s 3ms/step - loss: 0.7258 - accuracy: 0.5063
Epoch 10/30
1531/1531 [==============================] - 5s 3ms/step - loss: 0.7